<a href="https://colab.research.google.com/github/brew-brew-com/ML-Prep/blob/main/42_Parameter_Tuning_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
dataset = load_breast_cancer()

X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

In [3]:
# "est"、"pca"などの記号はグリッドサーチ時の設定に使用される

pipe_logistic = Pipeline([('scl', StandardScaler()), ('pca', PCA(random_state=1)), ('est', LogisticRegression(random_state=1))])

# パラメータグリッドの設定
# 記号 + "__" + パラメーター名　がお約束

# 主成分圧縮：抽出主成分数を5,7,9の3通り
# ロジスティック回帰：正則化パラメータは0.1から100の4通り、正則化はL1とL2の2通り
# 計24通り

param_grid_logistic = {'pca__n_components':[5, 7, 9], 'est__C':[0.1, 1.0, 10.0, 100.0], 'est__penalty':['l1', 'l2']}

display('探索空間:%s' %param_grid_logistic)

gs = GridSearchCV(estimator=pipe_logistic, param_grid=param_grid_logistic, scoring='f1', cv=3, return_train_score=False)
# gs = gs.fit(X, y.as_matrix().ravel())
gs = gs.fit(X, y)

# 探索した結果のベストスコアとパラメータの取得

print('Best Score:', gs.best_score_)
print('Best Params', gs.best_params_)
display(pd.DataFrame(gs.cv_results_))

"探索空間:{'pca__n_components': [5, 7, 9], 'est__C': [0.1, 1.0, 10.0, 100.0], 'est__penalty': ['l1', 'l2']}"

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_se

Best Score: 0.9819333023291866
Best Params {'est__C': 1.0, 'est__penalty': 'l2', 'pca__n_components': 9}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_est__C,param_est__penalty,param_pca__n_components,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005491,0.000569,0.000000,0.000000,0.1,l1,5,"{'est__C': 0.1, 'est__penalty': 'l1', 'pca__n_...",NaN,NaN,NaN,NaN,NaN,24
1,0.004934,0.000051,0.000000,0.000000,0.1,l1,7,"{'est__C': 0.1, 'est__penalty': 'l1', 'pca__n_...",NaN,NaN,NaN,NaN,NaN,22
2,0.007181,0.001736,0.000000,0.000000,0.1,l1,9,"{'est__C': 0.1, 'est__penalty': 'l1', 'pca__n_...",NaN,NaN,NaN,NaN,NaN,18
3,0.014796,0.002515,0.003009,0.000080,0.1,l2,5,"{'est__C': 0.1, 'est__penalty': 'l2', 'pca__n_...",0.979424,0.975410,0.978903,0.977912,0.001782,5
4,0.012774,0.000279,0.003165,0.000151,0.1,l2,7,"{'est__C': 0.1, 'est__penalty': 'l2', 'pca__n_...",0.983471,0.975410,0.983051,0.980644,0.003705,2
5,0.016433,0.002177,0.003232,0.000284,0.1,l2,9,"{'est__C': 0.1, 'est__penalty': 'l2', 'pca__n_...",0.983471,0.971429,0.983193,0.979364,0.005613,4
6,0.006675,0.000600,0.000000,0.000000,1,l1,5,"{'est__C': 1.0, 'est__penalty': 'l1', 'pca__n_...",NaN,NaN,NaN,NaN,NaN,17
7,0.005666,0.000146,0.000000,0.000000,1,l1,7,"{'est__C': 1.0, 'est__penalty': 'l1', 'pca__n_...",NaN,NaN,NaN,NaN,NaN,13
8,0.005867,0.000312,0.000000,0.000000,1,l1,9,"{'est__C': 1.0, 'est__penalty': 'l1', 'pca__n_...",NaN,NaN,NaN,NaN,NaN,23
9,0.014237,0.003345,0.003108,0.000474,1,l2,5,"{'est__C': 1.0, 'est__penalty': 'l2', 'pca__n_...",0.983333,0.983471,0.974359,0.980388,0.004263,3


In [5]:
# ベストモデルで予測

X_pred = gs.predict_proba(X)

print(gs.predict_proba(X))
print(gs.predict(X))

[[9.99999991e-01 9.16949126e-09]
 [9.99858219e-01 1.41781006e-04]
 [9.99999722e-01 2.78322042e-07]
 ...
 [9.94479111e-01 5.52088883e-03]
 [1.00000000e+00 2.00899844e-11]
 [7.89925108e-06 9.99992101e-01]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0